In [ ]:
import json
import time

fd = open("record_data.json", 'r')
txt = fd.read()
fd.close()
records1 = json.loads(txt)

print("Welcome to Online Electronics Store - By \"Renu Balhara\":)\n")

choice = input("Ready to shop? (Enter Y for yes and N for no)")
while choice == 'Y':
    print("1: Show all Categories")
    print("2: Exit")
    Categories = ['Mobiles', 'Laptops', 'Headphones', 'Cameras']
    response = int(input("Enter your Choice: "))
    if response == 1:
        # show all categories
        print("Category 1. Mobile \nCategory 2. Laptops \nCategory 3. Headphones \nCategory 4. Cameras")
        flag = True
        while flag:
            ch = int(input("What you are looking for ? (Enter category number) - "))
            if ch in range(1,5):
                flag = False
                val = 1001
                for i in range(30):
                    if records1[str(val)]['Product_Category'] == Categories[ch-1]:
                        print("ID -",str(val), end = '\t')
                        print("Name -",records1[str(val)]['Product_Name'], end = '\t')
                        print("Price -",records1[str(val)]['Product_Price'], end = '\t')
                        print("Discount -",records1[str(val)]["Product_Discount('%')"], end = '%\t')
                        if int(records1[str(val)]['Product_Quantity']) == 0:
                            print("Out of stock!!")
                        else:
                            print("Count left -",records1[str(val)]['Product_Quantity'])
                    val += 1
                    
                diff = input("Want to Choose different Category ? (Enter Y/N) - ")
                if(diff == 'Y'):
                    flag = True
                    
            else:
                print("Invalid choice please try again!!")
        
        # Ask for what to shop
        askForShop = input("Want to select products to shop ? (Y/N) ")
        if askForShop == 'Y':
            shopFlag = True
            shopAmount = 0.0
            totalDiscount = 0.0
            shopCart = []
            while shopFlag:
                prod_id = input("Enter product ID - ")
                if int(records1[prod_id]['Product_Quantity']) <= 0:
                    print("Sorry !! This product is currently out of stock :(")
                    continue
                quant = int(input("Enter quantity to purchase - "))
                while int(records1[prod_id]['Product_Quantity']) < quant or quant < 1:
                    print("Invalid Quantity!!")
                    quant = int(input("Enter quantity to purchase - "))
                amount = int(records1[prod_id]['Product_Price']) * quant;
                records1[prod_id]['Product_Quantity'] = str(round(int(records1[prod_id]['Product_Quantity']) - quant))
                totalDiscount += amount * ((int(records1[prod_id]["Product_Discount('%')"]))/100)
                amount -= (amount * ((int(records1[prod_id]["Product_Discount('%')"]))/100))
                shopAmount += amount
                shopCart.append([prod_id, quant])
                
                print("Product -", records1[prod_id]['Product_Name'], "added to Cart successfully.")
                print("Choose next step - ")
                nextStep = int(input("1.Proceed to pay. 2.Add more Products 3.Exit and cancel shopping"))
                if nextStep == 1:
                    shopFlag = False
                elif nextStep == 2:
                    shopFlag = True
                else:
                    # redo changes in records after cancellation
                    n = len(shopCart)
                    for i in range(n):
                        id = shopCart[i][0]
                        records1[id]['Product_Quantity'] = str(round(int(records1[id]['Product_Quantity']) + shopCart[i][1]))
                    break
            
            if(shopFlag):
                print("Cancellation successfull")
                break
            else:
                
                # get customer details
                name = input("Enter your name - ")
                phone_no = input("Enter Phone number - ")
                address = input("Enter address - ")
                paymentMode = input("Payment Mode (Paytm/PhonePe/NEFT) - ")
                purchase_time = time.ctime()
                
                # update the sales.json file
                sales_fd = open("sales.json", 'r')
                txt1 = sales_fd.read()
                sales_fd.close()
                sales_dict = json.loads(txt1)
                
                sales_dict['total sales'] = str(round(float(sales_dict['total sales']) + shopAmount, 3))
                sales_dict['Total Pruchases'] = str(int(sales_dict['Total Pruchases']) + 1)
                
                customer_id = str(10000 + int(sales_dict['Total Pruchases']))
                
                bill_dict = {}
                for i in range(len(shopCart)):
                    id = shopCart[i][0]
                    lst1 = {"ID": id, "Quantity": str(shopCart[i][1])}
                    bill_dict[str(i+1)] = lst1
                
                sales_dict[customer_id] = { "name": name, 
                                              "phone number": phone_no, 
                                              'address': address, 
                                              'payment mode': paymentMode, 
                                              'purchase time': purchase_time, 
                                              'bill': bill_dict, 
                                              "Total Amount": str(round(shopAmount, 3))
                                            }
                
                sales_fd = open("sales.json", 'w')
                sales_content = json.dumps(sales_dict)
                sales_fd.write(sales_content)
                sales_fd.close()
                
                # print the bill
                print("\n**********SHOPPING BILL**********\n")
                n = len(shopCart)
                for i in range(n):
                    id = shopCart[i][0]
                    print("Product " + str(i+1) + ": ID = " + id + " Name = " + records1[id]['Product_Name'] + 
                          " Quantity = " + str(shopCart[i][1]) + " Price = " + records1[id]['Product_Price'] +
                          " Discount = " + records1[id]["Product_Discount('%')"] + "%")
                
                print("Total Amount Saved - " + str(totalDiscount))
                print("Final Amount Paid - " + str(round(shopAmount, 3)))
                print("\n\n")
    else:
        break

print("\nThank you for visiting. Have a good day :)")

fd = open("record_data.json", 'w')
js = json.dumps(records1)
fd.write(js)
fd.close()